In [1]:
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
import json
import hashlib
import time
from random import randint
from time import sleep
import re
from fp.fp import FreeProxy
from lxml.html import fromstring
from itertools import cycle
import traceback

In [2]:
# proxies = list()
# for i in range(10):
#     proxy = FreeProxy().get()
#     proxies.append(proxy)

In [3]:
# def get_proxies():
#     url = 'https://free-proxy-list.net/'
#     response = requests.get(url)
#     parser = fromstring(response.text)
#     proxies = set()
#     for i in parser.xpath('//tbody/tr')[:10]:
#         if i.xpath('.//td[7][contains(text(),"yes")]'):
#             #Grabbing IP and corresponding PORT
#             proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
#             proxies.add(proxy)
#     return proxies

In [4]:
# proxy_pool = cycle(proxies)
# url = 'https://httpbin.org/ip'
# for i in range(1,11):
#     #Get a proxy from the pool
#     proxy = next(proxy_pool)
#     print("Request #%d"%i)
#     try:
#         response = requests.get(url,proxies={"http": proxy, "https": proxy})
#         print(response.json())
#     except:
#     #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
#     #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
#         print("Skipping. Connnection error")

In [5]:
headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Accept-Language': 'he-IL,he;q=0.9,en-US;q=0.8,en;q=0.7',
}


def load_soup_object(url_path):
    sleep(randint(3,5))
    page = requests.get(url_path, headers = headers)
    #print(page)
    return BeautifulSoup(page.content, "html.parser")

In [6]:
## name, current alias, affiliation, marital status, gender,
## height, weight, hair color, origin, living status, cause of death, reality, place of birth,
## identity, citenzenship, education, creators, unusual features, occupation


def time_convert(sec):
    mins = sec // 60
    sec = sec % 60
    hours = mins // 60
    mins = mins % 60
    print("Time Lapsed = {0}:{1}:{2}".format(int(hours),int(mins),sec))
    

In [7]:
dataframe = pd.DataFrame(columns = ['Name','Marital Status', 'Gender', 'Hair Color',
                                    'Eye Color', 'Living status', 'Reality','Identity',
                                    'Citenzenship','Appearnces', 'Year'])

name_lst = [] 
marital_status_lst = [] 
gender_lst = []
hair_color_lst = [] 
eye_color_lst = [] 
living_status_lst = []
reality_lst = []
identity_lst = [] 
citenzenship_lst = [] 
appearnces_lst = []
year_lst = []

url_prefix = 'https://marvel.fandom.com'
first_page = True

start_time = time.time()

In [ ]:
soup = load_soup_object('https://marvel.fandom.com/wiki/Category:Characters')
not_last_page = True
while not_last_page:
    current_page_character_list = soup.find_all('div', {'class': 'category-page__members-wrapper'})
    if first_page:
        links = soup.find_all('a',  {'class': 'category-page__member-link'})[18:]
        first_page = False
    else:
        links = soup.find_all('a',  {'class': 'category-page__member-link'})

    for i,X in enumerate(links):     
        #print(str(i) + ". " + url_prefix + X.get('href'))
        try:
            character_page_soup = load_soup_object(url_prefix + X.get('href'))
        except:
            print("An exception occurred during loading character page")
            #pass
        if character_page_soup.find('div',  {'data-source': 'Name'}) is not None:
            if character_page_soup.find('div',  {'data-source': 'Name'}).find('div',  {'class': 'pi-data-value pi-font'}).a is not None:
                name = character_page_soup.find('div',  {'data-source': 'Name'}).find('div',  {'class': 'pi-data-value pi-font'}).a
                name_lst.append(name.get_text().strip())
            else:
                name = character_page_soup.find('div',  {'data-source': 'Name'}).find('div',  {'class': 'pi-data-value pi-font'})
                name.get_text().strip()
                name_lst.append(name.get_text().strip())
        else:
            name_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'MaritalStatus'}) is not None:
            if character_page_soup.find('div', {'data-source': 'MaritalStatus'}).a is not None: 
                m_status = character_page_soup.find('div', {'data-source': 'MaritalStatus'}).a
                marital_status_lst.append(m_status.get_text().strip())
            else:
                marital_status_lst.append(None)
        else:
            marital_status_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'Reality'}) is not None: 
            reality_content = character_page_soup.find('div',  {'data-source': 'Reality'}).find('div',  {'class': 'pi-data-value pi-font'}).get_text().strip()
            reality_lst.append(reality_content)
        else:
            reality_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'Gender'}) is not None:
            if character_page_soup.find('div', {'data-source': 'Gender'}).a is not None:
                gender = character_page_soup.find('div', {'data-source': 'Gender'}).a
                gender_lst.append(gender.get_text().strip())
            else:
                gender_lst.append(None)
        else:
            gender_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'Hair'}) is not None:
            if character_page_soup.find('div', {'data-source': 'Hair'}).a is not None:
                hair = character_page_soup.find('div', {'data-source': 'Hair'}).a
                hair_color_lst.append(hair.get_text().strip())
            else:
                hair_color_lst.append(None)
        else:
            hair_color_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'Eyes'}) is not None:
            if character_page_soup.find('div', {'data-source': 'Eyes'}).a is not None:
                eye = character_page_soup.find('div', {'data-source': 'Eyes'}).a
                eye_color_lst.append(eye.get_text().strip())
            else:
                eye_color_lst.append(None)
        else:
            eye_color_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'Status'}) is not None:
            status = character_page_soup.find('div',{'data-source': 'Status'}).find_all('a')
            living_status_lst.append(status[0].get_text().strip())
        else:
            living_status_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'Identity'}) is not None:
            if character_page_soup.find('div',  {'data-source': 'Identity'}).a is not None:
                identity =  character_page_soup.find('div',  {'data-source': 'Identity'}).a.get_text().strip()
                identity_lst.append(identity)
            else:
                identity_lst.append(None)
        else:
            identity_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'Citizenship'}) is not None:
            citizen_content = character_page_soup.find('div',  {'data-source': 'Citizenship'}).find('div',  {'class': 'pi-data-value pi-font'}).get_text().strip()
            citenzenship_lst.append(citizen_content)
        else:
            citenzenship_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'First'}) is not None:
            year_content = character_page_soup.find('div', {'data-source': 'First'}).find_all('a')
            if(len(year_content) > 1):
                year = character_page_soup.find('div', {'data-source': 'First'}).find_all('a')[1].get_text().strip().split()[-1]
            else:
                if len(character_page_soup.find('div', {'data-source': 'First'}).get_text().strip().split()) > 0:
                    year = character_page_soup.find('div', {'data-source': 'First'}).get_text().strip().split()[-1]
                    if 'Mentioned' in year:
                        year = year[0:4]
                    if 'Carving' in year:
                        year = year[0:4]
                    if 'Series' in year:
                        year = year[0:4]
                    else:
                        year = character_page_soup.find('div', {'data-source': 'First'}).get_text().strip().split()[-1][:-1]
                else:
                    year = character_page_soup.find('div', {'data-source': 'First'}).get_text().strip()
            year_lst.append(year)
        else:
            year_lst.append(None)


        if character_page_soup.find('span',  {'id': 'See_Also'}) is not None:
            appearnces_parent = character_page_soup.find('span', {'id': 'See_Also'}).parent
            if appearnces_parent.next_sibling.next_sibling.find('li') is not None:
                appearnces_content = appearnces_parent.next_sibling.next_sibling.find('li').get_text().strip().split()[0]
                appearnces_lst.append(appearnces_content.split()[0])
            else:
                appearnces_lst.append(None)
        else:
            appearnces_lst.append(None)
       
    if soup.find('a', {'class':'category-page__pagination-next'}) is not None:
        next_page = soup.find('a', {'class':'category-page__pagination-next'}).get('href')
        try:
            soup = load_soup_object(next_page)
        except:
            print("An exception occurred during loading next page")
            #pass    
    else:
         not_last_page = False


An exception occurred during loading character page
An exception occurred during loading character page
An exception occurred during loading character page
An exception occurred during loading character page


In [ ]:
end_time = time.time()
time_lapsed = end_time - start_time
time_convert(time_lapsed)

In [ ]:
dataframe['Name'] = name_lst

In [ ]:
dataframe['Marital Status'] = marital_status_lst

In [ ]:
dataframe['Gender'] = gender_lst

In [ ]:
dataframe['Hair Color'] = hair_color_lst

In [ ]:
dataframe['Living status'] = living_status_lst

In [ ]:
dataframe['Reality'] = reality_lst

In [ ]:
dataframe['Identity'] = identity_lst

In [ ]:
dataframe['Citenzenship'] = citenzenship_lst

In [ ]:
dataframe['Eye Color'] = eye_color_lst

In [ ]:
dataframe['Appearnces'] = appearnces_lst

In [ ]:
dataframe['Year'] = year_lst

In [ ]:
dataframe

In [ ]:
dataframe.to_csv('Marvel Dataframe.csv')